In [2]:
!pip install psycopg2
import psycopg2
import pandas as pd

# PostgreSQL credentials
dbname = 'bibmet'
user = 'bibmetuser'
password = 'PASSWORD'
host = 'metrics.ub.gu.se'
port = '5432'

# Create a connection
conn = psycopg2.connect(dbname=dbname, user=user, password=password, host=host, port=port)

In [3]:
# Establishing a connection to the PostgreSQL database
with psycopg2.connect(dbname=dbname, user=user, password=password, host=host, port=port) as conn:
    # Creating a cursor to execute SQL queries
    with conn.cursor() as cursor:
        # Example query to retrieve data from the database
        query = """
SELECT * FROM

(SELECT distinct p.id AS pubid,f.name_sv AS fakultet,(select di.name_sv from departments di where di.id = d.inst_id) AS Institution, pv.pubyear, pt.label_sv AS dokumenttyp, pv.ref_value
FROM publications p
JOIN publication_versions pv ON pv.id = p.current_version_id
JOIN publication_types pt ON pv.publication_type_id=pt.id
JOIN people2publications p2p ON p2p.publication_version_id=pv.id
JOIN people pe ON pe.id=p2p.person_id
JOIN departments2people2publications d2p2p ON d2p2p.people2publication_id=p2p.id
JOIN departments d ON d.id=d2p2p.department_id
JOIN faculties f ON f.id=d.faculty_id
WHERE p.deleted_at IS NULL
AND pv.pubyear BETWEEN 2017 AND 2022
AND (p.process_state NOT IN ('DRAFT', 'PREDRAFT') OR p.process_state IS NULL)) tbl1
 LEFT OUTER JOIN

--matcha mot wos
(SELECT distinct p.id AS wos, (CASE 
          WHEN im.oa_field LIKE '%gold%' THEN 1
          WHEN im.oa_field LIKE '%green%' THEN 1
          WHEN im.oa_field LIKE '%Green%' THEN 1
          WHEN im.oa_field LIKE '%hybrid%' THEN 1
          WHEN im.oa_field LIKE '%Gold%' THEN 1
          WHEN im.oa_field LIKE '%Hybrid%' THEN 1
          WHEN im.oa_field LIKE '%bronze%' THEN 1
          WHEN im.oa_field LIKE '%Bronze%' THEN 1
          WHEN im.oa_field LIKE '%Hybrid%' THEN 1
          
      END) AS OA
 
FROM publications p
JOIN publication_versions pv ON pv.id = p.current_version_id
JOIN publication_types pt ON pv.publication_type_id=pt.id
JOIN people2publications p2p ON p2p.publication_version_id=pv.id
JOIN people pe ON pe.id=p2p.person_id
JOIN departments2people2publications d2p2p ON d2p2p.people2publication_id=p2p.id
JOIN departments d ON d.id=d2p2p.department_id
JOIN faculties f ON f.id=d.faculty_id
JOIN "cross".gup2ext g2 ON g2.pubid = p.id
JOIN "extra".isi_main im ON im.isi_id=g2.isi_id
WHERE p.deleted_at IS NULL
AND pv.pubyear BETWEEN 2017 AND 2022
AND (p.process_state NOT IN ('DRAFT', 'PREDRAFT') OR p.process_state IS NULL)
) tbl2
On tbl1.pubid=tbl2.wos

LEFT OUTER JOIN

(SELECT distinct p.id as cwts, fd."n*%", CASE when fd."fc*s"=0 then null
else fd."c*" / fd."fc*s" 
END AS cf
FROM publications p
JOIN publication_versions pv ON pv.id = p.current_version_id
JOIN publication_types pt ON pv.publication_type_id=pt.id
JOIN people2publications p2p ON p2p.publication_version_id=pv.id
JOIN people pe ON pe.id=p2p.person_id
JOIN departments2people2publications d2p2p ON d2p2p.people2publication_id=p2p.id
JOIN departments d ON d.id=d2p2p.department_id
JOIN faculties f ON f.id=d.faculty_id
JOIN "cross".gup2ext g2 ON g2.pubid = p.id
JOIN cwts.fielddata fd ON fd.isi_id = g2.isi_id
WHERE p.deleted_at IS NULL
AND fd.year=6
AND fd.cwtsyear BETWEEN 2017 AND 2022
AND (p.process_state NOT IN ('DRAFT', 'PREDRAFT') OR p.process_state IS NULL)) tbl3
On tbl1.pubid=tbl3.cwts

LEFT OUTER JOIN
--internationell publ
(SELECT distinct p.id AS international 
 --CASE when p.id>= 10 then 1 END
FROM extra.isi_main im
JOIN extra.isi_address ia ON ia.isi_id=im.isi_id
JOIN "cross".gup2ext g2e ON g2e.isi_id=im.isi_id
JOIN publications p ON g2e.pubid=p.id
JOIN publication_versions pv ON pv.id=p.current_version_id
JOIN people2publications p2p ON p2p.publication_version_id=pv.id
JOIN departments2people2publications d2p2p ON d2p2p.people2publication_id=p2p.id
JOIN departments d ON d.id=d2p2p.department_id
JOIN people pe ON pe.id=p2p.person_id
JOIN identifiers i ON i.person_id=pe.id
WHERE p.deleted_at IS NULL
AND (p.process_state NOT IN ('DRAFT', 'PREDRAFT') OR p.process_state IS NULL)
AND i.source_id = 1
AND pv.pubyear BETWEEN 2017 AND 2022
AND p.id in (
	SELECT p.id
	FROM publications p
	JOIN publication_versions pv ON pv.id=p.current_version_id
	JOIN people2publications p2p ON p2p.publication_version_id=pv.id
	JOIN departments2people2publications d2p2p ON d2p2p.people2publication_id=p2p.id
	JOIN departments d ON d.id=d2p2p.department_id
	WHERE p.deleted_at IS NULL
	AND (p.process_state NOT IN ('DRAFT', 'PREDRAFT') OR p.process_state IS NULL)
	AND (d.id = 666)
	AND pv.pubyear BETWEEN 2017 AND 2022
)
AND im.isi_id IN (
	SELECT isi_id FROM extra.isi_address WHERE country != 'Sweden'))tbl4

On tbl1.pubid=tbl4.international

LEFT OUTER JOIN
(SELECT distinct p.id as external
FROM publications p
JOIN publication_versions pv ON pv.id = p.current_version_id
JOIN publication_types pt ON pv.publication_type_id=pt.id
JOIN people2publications p2p ON p2p.publication_version_id=pv.id
JOIN people pe ON pe.id=p2p.person_id
JOIN departments2people2publications d2p2p ON d2p2p.people2publication_id=p2p.id
JOIN departments d ON d.id=d2p2p.department_id
JOIN faculties f ON f.id=d.faculty_id
WHERE p.deleted_at IS NULL
AND pv.pubyear BETWEEN 2017 AND 2022
AND (p.process_state NOT IN ('DRAFT', 'PREDRAFT') OR p.process_state IS NULL)
AND p.id IN ( 
  SELECT p1.id 
  FROM publications p1
  JOIN publication_versions pv1 ON pv1.id=p1.current_version_id
  JOIN people2publications p2p1 ON p2p1.publication_version_id=pv1.id
  JOIN departments2people2publications d2p2p1 ON d2p2p1.people2publication_id=p2p1.id
  WHERE d2p2p1.department_id = '666')
) tbl5

On tbl1.pubid=tbl5.external

        """
  # Fetching the results into a Pandas DataFrame
        df = pd.read_sql(query, conn)

# Display the DataFrame
df.head()

C:\Users\xkulll\AppData\Local\Temp\ipykernel_18388\2655252211.py:135: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  df = pd.read_sql(query, conn)


,pubid,fakultet,institution,pubyear,dokumenttyp,ref_value,wos,oa,cwts,n*%,cf,international,external
0,120600,Utbildningsvetenskapliga fakulteten,"Institutionen för pedagogik, kommunikation och...",2018,Kapitel i bok,NOTREF,NaN,NaN,NaN,NaN,NaN,NaN,120600.0
1,120600,Utbildningsvetenskapliga fakulteten,Linnécentret for forskning om lärande (LinCS),2018,Kapitel i bok,NOTREF,NaN,NaN,NaN,NaN,NaN,NaN,120600.0
2,158757,Handelshögskolan,Gothenburg Research Institute (GRI),2020,Kapitel i bok,NOTREF,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,205543,Naturvetenskapliga fakulteten,Institutionen för matematiska vetenskaper,2017,Artikel i vetenskaplig tidskrift,ISREF,NaN,NaN,NaN,NaN,NaN,NaN,205543.0
4,205684,Handelshögskolan,Företagsekonomiska institutionen,2017,Artikel i vetenskaplig tidskrift,ISREF,205684.0,NaN,205684.0,0.0,0.73377,NaN,NaN


In [15]:
HH = pivot_df[pivot_df['institution'].isin(['Centrum för globalisering och utveckling (GCGD)', 'Centrum för finans', 'Institutionen för nationalekonomi med statistik', 'Institutionen för ekonomi och samhälle',' Juridiska institutionen', 'Centrum för hälsoekonomi (CHEGU)', 'Företagsekonomiska institutionen', 'Gothenburg Research Institute (GRI)', 'Centrum för konsumtionsvetenskap (CFK)', 'Environment for Development (EfD)'])]
HF = pivot_df[pivot_df['institution'].isin(['Centrum för Digital Humaniora (CDH)', 'Institutionen för filosofi, lingvistik och vetenskapsteori', 'Institutionen för historiska studier', 'Institutionen för kulturvetenskaper', 'Institutionen för litteratur, idéhistoria och religion', 'Institutionen för språk och litteraturer', 'Institutionen för svenska språket', 'Institutionen för svenska, flerspråkighet och språkteknologi'])]
IT = pivot_df[pivot_df['institution'].isin(['Institutionen för data- och informationsteknik (GU)', 'Institutionen för tillämpad informationsteknologi (GU)', 'Kollegium SSKKII (2010-)'])]
KF = pivot_df[pivot_df['institution'].isin(['Akademin Valand', 'Högskolan för design och konsthantverk',' Högskolan för scen och musik', 'HDK-Valand - Högskolan för konst och design'])]
NF = pivot_df[pivot_df['institution'].isin(['Institutionen för biologi och miljövetenskap', 'Institutionen för fysik (GU)', 'Institutionen för geovetenskaper', 'Institutionen för kemi och molekylärbiologi', 'Institutionen för kulturvård', 'Institutionen för marina vetenskaper', 'Institutionen för matematiska vetenskaper', 'Sven Lovén centrum för marina vetenskaper', 'Svenskt NMR-centrum vid Göteborgs universitet'])]	
SA = pivot_df[pivot_df['institution'].isin(['Institutionen för biomedicin', 'Institutionen för kliniska vetenskaper' , 'Institutionen för medicin', 'Institutionen för neurovetenskap och fysiologi', 'Institutionen för odontologi', 'Institutionen för vårdvetenskap och hälsa'])]
SF = pivot_df[pivot_df['institution'].isin(['Centrum för Europaforskning (CERGU)', 'Förvaltningshögskolan', 'Institutionen för globala studier', 'Institutionen för journalistik, medier och kommunikation (JMG)', 'Institutionen för socialt arbete', 'Institutionen för sociologi och arbetsvetenskap', 'Psykologiska institutionen', 'Statsvetenskapliga institutionen'])]
UF = pivot_df[pivot_df['institution'].isin(['Institutionen för didaktik och pedagogisk profession', 'Institutionen för kost- och idrottsvetenskap', 'Institutionen för pedagogik och specialpedagogik', 'Institutionen för pedagogik, kommunikation och lärande', 'Linnécentret for forskning om lärande (LinCS)'])]

In [30]:

!pip install reportlab
import pandas as pd
from reportlab.lib.pagesizes import letter, inch
from reportlab.platypus import SimpleDocTemplate, Paragraph, Table, TableStyle, Spacer, PageBreak, Image
from reportlab.lib.styles import getSampleStyleSheet
from reportlab.lib import colors
from PIL import Image as PILImage
from datetime import datetime


# Create a PDF document
pdf_filename = 'bibliometrisk_rapport.pdf'
document = SimpleDocTemplate(pdf_filename, pagesize=letter)

# Define styles
styles = getSampleStyleSheet()

# Function to create a table with styling
def create_table(data, col_widths, section_title, introduction_text, short_text):
    table_data = [data.columns.tolist()] + data.values.tolist()
    table = Table(table_data, colWidths=col_widths)
    table.setStyle(TableStyle([
        ('BACKGROUND', (0, 0), (-1, 0), colors.grey),
        ('TEXTCOLOR', (0, 0), (-1, 0), colors.whitesmoke),
        ('ALIGN', (0, 0), (0, -1), 'LEFT'),  # Align text in the first column to the left
        ('ALIGN', (1, 0), (-1, -1), 'CENTER'),  # Align text in other columns to the center
        ('FONTNAME', (0, 0), (-1, 0), 'Helvetica-Bold'),
        ('BOTTOMPADDING', (0, 0), (-1, 0), 12),
        ('BACKGROUND', (0, 1), (-1, -1), colors.white),
        ('GRID', (0, 0), (-1, -1), 1, colors.black),
        ('FONTSIZE', (0, 0), (-1, -1), 9),
    ]))
    
    intro_paragraph = Paragraph(introduction_text, styles['Normal'])
    short_text_paragraph = Paragraph(short_text, styles['Normal'])
    
    return [Paragraph(section_title, styles['Heading1']), Spacer(1, 12), intro_paragraph, Spacer(1, 6), short_text_paragraph, Spacer(1, 12), table]
    

# Content for the report
title = "BIBLIOMETRISK RAPPPORT 2023"
authors = "Digitala tjänster, Göteborgs universitetsbibliotek"
date = datetime.now().strftime("%Y-%m-%d")

# Logo path (replace 'path/to/logo.png' with the actual path to your logo file)
logo_path = 'C:/users/username/desktop/logga.jpg'

# Add data to content
content = [
    # Logo on the front page
    Image(logo_path, width=2 * inch),
    
    # Title, authors, and date
    Spacer(1, 24),  # Add space between logo and text
    Paragraph(title, styles['Title']),
    Spacer(1, 12),
    Paragraph(authors, styles['Normal']),
    Spacer(1, 12),
    Paragraph(date, styles['Normal']),
    PageBreak(),
]

# Methodology section
methodology_title = "Introduction"
methodology_text = """
This section describes the methodology used for the bibliometric analysis.
Provide details on the data sources, analysis techniques, and any other relevant information.
Adjust the text size as needed.
"""

# Add data to content
content = [
    # Logo on the front page
    Image(logo_path, width=3 * inch, height=1 * inch),
    # Increased space between logo and title
    Spacer(1, 48),
    
    # Title, authors, and date
    Paragraph(title, styles['Title']),
    Spacer(1, 400),
    Paragraph(authors, styles['Normal']),
    Spacer(1, 12),
    Paragraph(date, styles['Normal']),
    PageBreak(),
    
    # Methodology section
    Paragraph(methodology_title, styles['Heading1']),
    Spacer(1, 12),
    Paragraph(methodology_text, styles['Normal']),
    PageBreak(),  # Add a page break after the methodology section
]
column_widths = [4 * inch, "*"]

# Add Faculty 1
intro_text_1 = "Introduction to Faculty 1."
short_text_1 = "Short description of Faculty 1."
content.extend(create_table(HH, column_widths, "Handelshögskolan", intro_text_1, short_text_1))
content.append(PageBreak()) 

# Add Faculty 2
intro_text_2 = "Introduction to Faculty 2."
short_text_2 = "Short description of Faculty 2."
content.extend(create_table(HF, column_widths, "Humanistiska fakulteten", intro_text_2, short_text_2))
content.append(PageBreak()) 

# Add Faculty 3
intro_text_3 = "Introduction to Faculty 3."
short_text_3 = "Short description of Faculty 3."
content.extend(create_table(IT, column_widths, "IT-fakulteten", intro_text_3, short_text_3))
content.append(PageBreak()) 

# Add Faculty 4
intro_text_4 = "Introduction to Faculty 4."
short_text_4 = "Short description of Faculty 4."
content.extend(create_table(KF, column_widths, "Konstnärliga fakulteten", intro_text_4, short_text_4))
content.append(PageBreak()) 

# Add Faculty 5
intro_text_5 = "Introduction to Faculty 5."
short_text_5 = "Short description of Faculty 5."
content.extend(create_table(NF, column_widths, "Naturvetenskapliga fakulteten", intro_text_5, short_text_5))
content.append(PageBreak()) 

# Add Faculty 6
intro_text_6 = "Introduction to Faculty 6."
short_text_6 = "Short description of Faculty 6."
content.extend(create_table(SA, column_widths, "Sahlgrenska akademin", intro_text_6, short_text_6))
content.append(PageBreak()) 

# Add Faculty 7
intro_text_7 = "Introduction to Faculty 7."
short_text_7 = "Short description of Faculty 7."
content.extend(create_table(SF, column_widths, "Samhällsvetenskapliga fakulteten", intro_text_7, short_text_7))
content.append(PageBreak()) 

# Add Faculty 8
intro_text_8 = "Introduction to Faculty 8."
short_text_8 = "Short description of Faculty 8."
content.extend(create_table(UF, column_widths, "Utbildningsvetenskapliga fakulteten", intro_text_8, short_text_8))
content.append(PageBreak()) 

# Build the PDF document
document.build(content)


# Display the PDF in Jupyter Notebook (optional)
from IPython.display import display, FileLink
display(FileLink(pdf_filename))


C:\Users\username\bibliometrisk_rapport.pdf